# fz-Cast3m Example Usage

This notebook demonstrates how to use the fz-Cast3m plugin for parametric studies with Cast3m finite element software.

## Prerequisites

- fz framework installed: `pip install git+https://github.com/Funz/fz.git`
- Cast3m software installed (castem2000 or cast3m in PATH)
- This repository cloned

In [ ]:
import fz
import pandas as pd
import os

# Change to repository root if needed
if not os.path.exists('.fz'):
    os.chdir('..')

## 1. Parse Variables from Input File

First, let's examine what variables are in our parametric beam example:

In [ ]:
# Parse variables from input file
variables = fz.fzi('examples/Cast3m/poutre_parametric.dgibi', 'Cast3m')
print("Variables found in input file:")
for var, info in variables.items():
    print(f"  - {var}: {info}")

## 2. Compile Input File with Specific Values

Now let's compile the input file with specific parameter values:

In [ ]:
# Compile with specific values
fz.fzc(
    'examples/Cast3m/poutre_parametric.dgibi',
    {'long': 0.30, 'haut': 0.001, 'larg': 0.01, 'F': 1.0},
    'Cast3m',
    output_dir='compiled_example'
)

# Show the compiled file
import glob
compiled_files = glob.glob('compiled_example/**/poutre_parametric.dgibi', recursive=True)
if compiled_files:
    print("Compiled file:")
    with open(compiled_files[0]) as f:
        for i, line in enumerate(f, 1):
            print(f"{i:3d}: {line.rstrip()}")
            if i > 15:  # Show first 15 lines
                print("    ...")
                break

## 3. Run Parametric Study

Now let's run a parametric study with multiple parameter values.

**Note**: This requires Cast3m to be installed. If Cast3m is not available, this cell will fail.

In [ ]:
# Run parametric study
# We'll vary the length and force, keeping other parameters constant
results = fz.fzr(
    input_path='examples/Cast3m/poutre_parametric.dgibi',
    input_variables={
        'long': [0.20, 0.25, 0.30, 0.35, 0.40],  # 5 values
        'haut': 0.001,                            # constant
        'larg': 0.01,                             # constant
        'F': [0.5, 1.0, 1.5]                     # 3 values
    },
    model='Cast3m',
    calculators='localhost_Cast3m',
    results_dir='notebook_results'
)

print(f"\nTotal number of runs: {len(results)}")
print("\nFirst few results:")
results.head(10)

## 4. Analyze Results

Let's analyze the relationship between parameters and outputs:

In [ ]:
import matplotlib.pyplot as plt

# Plot displacement vs length for different forces
fig, ax = plt.subplots(figsize=(10, 6))

for force in results['F'].unique():
    data = results[results['F'] == force].sort_values('long')
    ax.plot(data['long'], data['dep_P2'], marker='o', label=f'F={force}')

ax.set_xlabel('Length (m)')
ax.set_ylabel('Displacement dep_P2')
ax.set_title('Beam Displacement vs Length for Different Forces')
ax.legend()
ax.grid(True)
plt.show()

## 5. Summary Statistics

Let's compute some summary statistics:

In [ ]:
print("Summary statistics for dep_P2:")
print(results.groupby('F')['dep_P2'].describe())

## Cleanup

Remove temporary files:

In [ ]:
import shutil

# Remove temporary directories
for directory in ['compiled_example', 'notebook_results']:
    if os.path.exists(directory):
        shutil.rmtree(directory)
        print(f"Removed {directory}")